# Toronto neighborhoods
In this notebook, I will explore and cluster the neighborhoods in Toronto

In [157]:
#importing necessary libs
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup as BS  #get website data
import requests
import unicodedata
print("Libs imported")

Libs imported


This wikipedia page has the postal code of some Neighborhoods of Toronto

The data is colected using the BeatifulSoup and requests libs, and transformed as a pandas DataFrame.

In [150]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
r = requests.get(url)
soup = BS(r.text)
#print(soup.prettify())

table = soup.find('table')
rows = table.find_all('tr')

l = []
for tr in rows:
    td = tr.find_all('td')
    thisRow = [tr.text for tr in td]
    thisRow = [x.encode('ascii','ignore') for x in thisRow]
    thisRow = [x.replace('\n','') for x in thisRow]
    l.append(thisRow)
    
data = pd.DataFrame(l,columns=["PostalCode","Borough","Neighborhood"])


Removing all rows with Not assigned Boroughs.

Also, setting the Not Assigned neighborhoods to match the borough

In [151]:
data = data[data.Borough != "Not assigned"]
data = data.iloc[1:,]
data.reset_index(inplace=True,drop=True)

In [155]:
print(data.iloc[6,:])
for i in range(data.shape[0]):
    if(data.iloc[i,2] == "Not assigned"):
        data.iloc[i,2] = data.iloc[i,1]
print(data.iloc[6,:])

PostalCode               M7A
Borough         Queen's Park
Neighborhood    Not assigned
Name: 6, dtype: object
PostalCode               M7A
Borough         Queen's Park
Neighborhood    Queen's Park
Name: 6, dtype: object


Some postalCodes have multiple lines in the dataset.

At these, the borough is always the same, but for some postal codes, there are multiple Neighborhoods.

Following, I will concatenate all the diferent neighborhoods with the same postal code in an unique row.

In [164]:
ll = []
postals = data["PostalCode"].unique()
for postal in postals:
    rows = data[data.PostalCode == postal]
    uniques = rows["Neighborhood"].unique()
    val = ', '.join(str(elem) for elem in uniques)
    borough = rows["Borough"].iloc[0]
    ll.append([postal,borough,val])

data = pd.DataFrame(ll,columns=["PostalCode","Borough","Neighborhood"])
print(data.shape)
print(data[data.PostalCode=="M5A"])

(103, 3)
  PostalCode           Borough               Neighborhood
2        M5A  Downtown Toronto  Harbourfront, Regent Park


Printing the shape and head of the result dataset, finishing the first part of the assignment

In [166]:
print(data.shape)
print(data.head(10))

(103, 3)
  PostalCode           Borough                      Neighborhood
0        M3A        North York                         Parkwoods
1        M4A        North York                  Victoria Village
2        M5A  Downtown Toronto         Harbourfront, Regent Park
3        M6A        North York  Lawrence Heights, Lawrence Manor
4        M7A      Queen's Park                      Queen's Park
5        M9A         Etobicoke                  Islington Avenue
6        M1B       Scarborough                    Rouge, Malvern
7        M3B        North York                   Don Mills North
8        M4B         East York   Woodbine Gardens, Parkview Hill
9        M5B  Downtown Toronto          Ryerson, Garden District
